In [207]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import warnings

warnings.filterwarnings('ignore')

In [208]:
folder_path = r'C:\Users\Rafael_Fagundes\Downloads\compiled_data.csv'

df = pd.read_csv(folder_path, encoding='utf-8')

df = df[
        (df['Country_x'] == 'United States')
       ]


In [209]:
grouped_df = df.groupby(['Fiscal Quarter','Fiscal Week','Audience Type','BU'])[['Spend', 'Net Rev']].sum().reset_index()

In [210]:
grouped_df.head()

,Fiscal Quarter,Fiscal Week,Audience Type,BU,Spend,Net Rev
0,2023-Q3,2023-W27,1PD,B2B,9.064157e+04,9.322798e+06
1,2023-Q3,2023-W27,1PD,CSB,5.925631e+05,1.721498e+06
2,2023-Q3,2023-W27,3PD,B2B,4.371169e+05,1.465023e+07
3,2023-Q3,2023-W27,3PD,CSB,1.528165e+06,1.249397e+06
4,2023-Q3,2023-W28,1PD,B2B,2.063858e+05,1.295381e+07


In [211]:
calendar_df = pd.read_csv(r'..\..\Dell-Projects\Helper\Calendar.csv', parse_dates=['Date'], encoding='utf-8')

In [212]:
calendar_df['Fiscal Week'] = calendar_df['FY'].str[-4:] + "-W" + calendar_df['WK'].astype(str).apply(lambda x: '0' + x if len(x) == 1 else x)

In [213]:
grouped_calendar_df = calendar_df.groupby(['Fiscal Week','QWK']).nunique().reset_index()

In [214]:
merged_df = pd.merge(grouped_df, grouped_calendar_df, on='Fiscal Week', how='left')

In [215]:
merged_df.head()

,Fiscal Quarter,Fiscal Week,Audience Type,BU,Spend,Net Rev,QWK,Date,Day,FY,Qtr,Month,Period,WK,FQ,FQW
0,2023-Q3,2023-W27,1PD,B2B,9.064157e+04,9.322798e+06,1,7,7,1,1,1,1,1,1,1
1,2023-Q3,2023-W27,1PD,CSB,5.925631e+05,1.721498e+06,1,7,7,1,1,1,1,1,1,1
2,2023-Q3,2023-W27,3PD,B2B,4.371169e+05,1.465023e+07,1,7,7,1,1,1,1,1,1,1
3,2023-Q3,2023-W27,3PD,CSB,1.528165e+06,1.249397e+06,1,7,7,1,1,1,1,1,1,1
4,2023-Q3,2023-W28,1PD,B2B,2.063858e+05,1.295381e+07,2,7,7,1,1,1,1,1,1,1


In [216]:
merged_df.to_csv(r'C:\Users\Rafael_Fagundes\Downloads\merged_df.csv')